In [1]:
import pandas as pd
import numpy as np
import ast

# %%
credits=pd.read_csv('Credits.csv',usecols=['movie_id','title','cast','crew'])
movies=pd.read_csv('Movies.csv')

# %%
credits.head(4)

# %%
movies.head(1)

# %%
# merging both files in one
movies=movies.merge(credits,left_on='title',right_on='title')

# %%
movies.head(1)

# %%
movies.head(1)

# %%
movies.rename(columns={'cast_x': 'cast', 'crew_y': 'crew', 'movie_id_x': 'movie_id'}, inplace=True)
movies.head(1)

# %%
# as data in genres and keywords are in objects we need to get to real data before processing it .
# this function extracts the name in the object passed.
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


# %%
movies['genres']=movies['genres'].apply(convert)



# %%
movies.head(1)

# %%
movies['keywords']=movies['keywords'].apply(convert)


# %%
movies.head(1)

# %%
movies['cast'][0]

# %%
movies=movies[movies.crew.notnull()]




# %%


# %%
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head(1)

# %%
lst=[]
for i in ast.literal_eval(movies['cast'][0]):
    lst.append(i['name'])
print(lst[:3])


# %%
def convertcast(obj):
    L=[]
    try:
        for i in ast.literal_eval(obj):
            L.append(i['name'])
        return L[:3]
    except:
        pass
movies['cast']=movies['cast'].apply(convertcast)

# %%
movies.head(1)

# %%
def convertcrew(obj):
    L=[]
    try:
        for i in ast.literal_eval(obj):
            if i['job']=="Director":

                L.append(i['name'])
        return L[:3]
    except:
        pass
movies['crew']=movies['crew'].apply(convertcrew)

# %%
movies.head(1)

# %%
movies['tags']=movies['genres']+movies['keywords']+movies['cast']+movies['crew']

# %%
movies['tags']

# %%
movies=movies[['movie_id','title','overview','tags']]

# %%
movies.dropna(inplace=True)

# %%
movies['tags']=movies['tags'].apply(lambda x: " ".join(x))

# %%
movies['tags']=movies['tags'].apply(lambda x: x.lower())

# %%
movies['tags']

# %%
movies.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_27512\2231828110.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  credits=pd.read_csv('Credits.csv',usecols=['movie_id','title','cast','crew'])


['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']


,movie_id,title,overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",action adventure fantasy science fiction cultu...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",adventure fantasy action ocean drug abuse exot...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,action adventure crime spy based on novel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,action crime drama thriller dc comics crime fi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",action adventure science fiction based on nove...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words="english")
tfidf_matrix=tfidf.fit_transform(movies['tags'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim= cosine_similarity(tfidf_matrix, tfidf_matrix)
def get_recommendation(title,cosine_sim=cosine_sim):
    idx=movies[movies['title']== title].index[0]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=sorted(sim_scores,key=lambda x: x[1], reverse=True)
    sim_scores=sim_scores[1:11]
    movie_indices=[i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

import pickle
with open('movie_data.pk1','wb') as file:
    pickle.dump((movies,cosine_sim),file)